In [5]:
#import fighter statistics
import pandas
import numpy
import math
pathname = "/Users/chugh/Documents/UFC Fight Code/Fighter Stats 62720 old.csv"
stats = pandas.read_csv(pathname, index_col = 0)
stats1 = stats.reset_index()

In [6]:
#import fighter matchups
pathname2 = "/Users/chugh/Documents/UFC Fight Code/Matchups 62720.csv"
matchups = pandas.read_csv(pathname2, index_col = 0)
matchups = matchups.reset_index()

In [9]:
#compare fighters and calculate their "scores"
def calc_pts(Fighter1,Fighter2,stats):
    pts = 0
    #significant strikes
    Fighter1_SSpM = stats.loc[Fighter1,"SSpM"]
    Fighter2_StrDef = stats.loc[Fighter2,"StrDef"]
    Fighter1_SSpM = Fighter1_SSpM*(1-Fighter2_StrDef)
    #only consider first three rounds since only two main events will have 5 possible
    pts = pts + Fighter1_SSpM*15*0.5
    
    #if the opponent throws a lot of takedowns and they land, this will reduce strikes
    Fighter2_TD = stats.loc[Fighter2, "TDpM"]
    Fighter1_TDDef = stats.loc[Fighter1,"TDDef"]
    Fighter2_TD = Fighter2_TD*(1-Fighter1_TDDef)
    pts = pts - Fighter2_TD*0.5 #assume that each takedown takes 1 minute off of significant strikes
    
    #takedowns - actually takedowns per 15 minutes
    Fighter1_TD = stats.loc[Fighter1, "TDpM"]
    Fighter2_TDDef = stats.loc[Fighter2,"TDDef"]
    Fighter1_TD = Fighter1_TD*(1-Fighter2_TDDef)
    pts = pts + Fighter1_TD*5
    
    #calculate pts based on rnd they would likely win
    Fighter1_rd1 = stats.loc[Fighter1,"1stRnd"]
    Fighter1_rd2 = stats.loc[Fighter1,"2ndRnd"]
    Fighter1_rd3 = stats.loc[Fighter1,"3rdRnd"]
    Fighter1_rd4 = stats.loc[Fighter1,"4thRnd"]
    Fighter1_rd5 = stats.loc[Fighter1,"5thRnd"]
    Fighter1_dec = stats.loc[Fighter1,"Dec"]
    pts = pts + (Fighter1_rd1*90 + Fighter1_rd2*70 + Fighter1_rd3*45 + Fighter1_rd4*40 + Fighter1_rd5*40 + Fighter1_dec*30)
    
    #may want to account for "durability" of opponent i.e. which rounds they typically lose in
    
    #model slow decline in performance after age 30
    Fighter1_Age = stats.loc[Fighter1, "Age"]
    if Fighter1_Age > 30:
        age_diff = Fighter1_Age - 30;
        age_fac = (1+math.exp(age_diff)*0.0015)
        pts = pts/age_fac
        
    
    return pts  



total_pts = []
for curr_fighter,row in matchups.iterrows():
    Fighter1 = row[0]
    Fighter2 = row[1]
    
    pts1 = calc_pts(Fighter1,Fighter2,stats)
    pts2 = calc_pts(Fighter2,Fighter1,stats) 
    
    #use rank to assess level of competition - still need to work on this part
    Fighter1_Rank = stats.loc[Fighter1, "Rank"]
    Fighter2_Rank = stats.loc[Fighter2,"Rank"]
    if Fighter1_Rank > Fighter2_Rank:
        pts1 = pts1*((Fighter2_Rank+1)/(Fighter1_Rank))**(1/5)
        print((((Fighter2_Rank+1)/(Fighter1_Rank)))**(1/5))
    if Fighter2_Rank > Fighter1_Rank:
        pts2 = pts2*(((Fighter1_Rank+1)/(Fighter2_Rank)))**(1/5)
        print((((Fighter1_Rank+1)/(Fighter2_Rank)))**(1/5))
        
        
    #modify based on number of opponent's fights in UFC
    Fighter1_FinUFC = stats.loc[Fighter1,"FinUFC"]
    Fighter2_FinUFC = stats.loc[Fighter2,"FinUFC"]
    fight_diff = Fighter1_FinUFC - Fighter2_FinUFC
    if fight_diff < 0:
        pts1 = pts1*(5/(5+math.log(abs(fight_diff))))
    if fight_diff > 0:
        pts2 = pts2*(5/(5+math.log(fight_diff)))
        
        
    #multiply by win-loss ratio of last 10 fights
    #Fighter1_WL = stats.loc[Fighter1, "WL"]
    #Fighter2_WL = stats.loc[Fighter2,"WL"]
    #WL_diff = Fighter1_WL - Fighter2_WL
    #if WL_diff > 0:
        #pts2 = pts2*(1-WL_diff)
    #if WL_diff < 0:
        #pts1 = pts1*(1-abs(WL_diff))
        
    #finally use opponents points to "z-score" point spread
    #pts1_final = pts1 - pts2
    #pts2_final = pts2 - pts1
        
        
    total_pts.append(pts1)
    total_pts.append(pts2)
    
#combine fighter names and pt values
total_pts = total_pts - min(total_pts) #so that bottom fighter is at 0
pt_series = pandas.Series(total_pts,name="Points")
Fighter_points = pandas.concat([stats1["Name"], pt_series], axis=1)
Fighter_points = Fighter_points.sort_values("Points", 0, ascending=False)
print(Fighter_points)

1.0
0.7885427284743957
0.9721786261445742
0.6488429169812621
0.9306474054037337
0.8153055965854519
1.0
0.8828798689141534
              Name     Points
1       Dan Hooker  36.678343
2       Mike Perry  35.539593
6    Brendan Allen  32.485843
0   Dustin Poirier  24.700082
12       Luis Pena  23.996343
14  Jordan Griffin  18.407343
15   Youssef Zalal  16.977430
8     Philipe Lins  13.384269
5   Maurice Greene  11.634499
10    Sean Woodson  10.025790
4    Gian Villante   7.381538
11    Julian Erosa   4.211461
13    Khama Worthy   2.227824
3      Mickey Gall   1.839610
7     Kyle Daukaus   0.775557
9     Tanner Boser   0.000000


In [3]:
# A Dynamic Programming based Python  
# Program for 0-1 Knapsack problem 
# Returns the maximum value that can  
# be put in a knapsack of capacity W 
def knapSack(W, wt, val, n): 
    fighter = []
    K = [[0 for x in range(W + 1)] for x in range(n + 1)] 
  
    # Build table K[][] in bottom up manner 
    for i in range(n + 1): 
        for w in range(W + 1): 
            if i == 0 or w == 0: 
                K[i][w] = 0
            elif wt[i-1] <= w: 
                K[i][w] = max(val[i-1] + K[i-1][w-wt[i-1]],  K[i-1][w]) 
            else: 
                K[i][w] = K[i-1][w] 
        fighter.append(K[i][w])
    return K[n][W], fighter
  
# Driver program to test above function 
opt, fighter_nums = knapSack(max_salary, all_sals, fight_pts, len(all_sals)) 
print(fighter_nums)

NameError: name 'max_salary' is not defined